In [18]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict    
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [ ]:
load_dotenv()  # Load environment variables from .env file


In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# 1. Set your NEW API key
os.environ["GOOGLE_API_KEY"] = 'AIzaSyBCkV3q0gebianxWAeQd-XXXXXXXXXXXXXXXXXXXXXX'

# 2. Use the correct model string for the current API
# Note: Using "gemini-1.5-flash" is the best path for LangGraph
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0
)

# 3. Test it before running the workflow
try:
    response = model.invoke("How far is the moon from the earth?")
    print(response.content)
except Exception as e:
    print(f"Error: {e}")

The Moon's distance from Earth isn't constant because it orbits in an ellipse, not a perfect circle.

Here are the key figures:

*   **Average distance:** Approximately **384,400 kilometers (238,900 miles)**
*   **Closest point (perigee):** Around **363,104 kilometers (225,622 miles)**
*   **Farthest point (apogee):** Around **405,696 kilometers (252,088 miles)**

So, while the average is often cited, the actual distance varies throughout the lunar month.


In [38]:
#create a state 
class LLMState(TypedDict):
    question:str
    anwser:str



In [39]:
def llm_qa(state:LLMState)->LLMState:

    #extract the questions from state
    question=state['question']

    #form the prompt
    prompt = f"Answer the following question: {question}"


    #ask the qustion to the model
    anwser=model.invoke(prompt).content


    #update the anwser in the state
    state['anwser']=anwser

    return state

In [40]:
graph=StateGraph(LLMState)
#add your nodes 
graph.add_node(llm_qa)

#add some edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)



workflow=graph.compile()

In [53]:
def llm_qa(state):
    question = state["question"]
    prompt = f"Answer the following question: {question}"

    answer = model.invoke(prompt).content
    state["answer"] = answer

    return state


In [57]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state)  # Output will include the answer from the LLM



{'question': 'How far is moon from the earth?', 'anwser': 'The distance between the Earth and the Moon is not constant because the Moon orbits the Earth in an elliptical path.\n\nHere are the key distances:\n\n*   **Average distance:** Approximately **384,400 kilometers (238,900 miles)**\n*   **Perigee (closest point):** Approximately **363,104 kilometers (225,622 miles)**\n*   **Apogee (farthest point):** Approximately **405,696 kilometers (252,088 miles)**\n\nSo, while the average is often cited, the actual distance varies throughout the lunar month.'}


In [55]:
model.invoke('How far is moon from the earth?').content

"The average distance from the Earth to the Moon is approximately **384,400 kilometers (km)**.\n\nThis is about **238,900 miles**.\n\nHowever, the Moon's orbit around the Earth is not a perfect circle; it's an ellipse. This means the distance varies throughout the month:\n\n*   **Perigee (closest point):** About **363,104 km (225,623 miles)**\n*   **Apogee (farthest point):** About **405,696 km (252,088 miles)**\n\nLight from the Moon takes about **1.28 seconds** to reach Earth."